In [ ]:
using JLD2
using MLJ

include("../src/reduction.jl")
include("../src/ensembles.jl")

@JLD2.load "variables/train_dataset.jld2" train_inputs train_targets
@JLD2.load "variables/test_dataset.jld2" test_inputs test_targets
@JLD2.load "variables/cross_val_index.jdl2" cross_val_index


### Aplicamos PCA sobre los folds

In [ ]:
folds = []
n_folds = unique(cross_val_index)
for fold in n_folds
    push!(folds, pca(train_inputs[findall(x -> x == fold, cross_val_index), :], 0.95)[3])
end;
n_comp = maximum(folds)

In [ ]:
BaggingClassifier = MLJ.@load BaggingClassifier pkg=MLJScikitLearnInterface verbosity=0
AdaBoost = MLJ.@load AdaBoostStumpClassifier pkg=DecisionTree verbosity=0
EvoTreeClassifier = MLJ.@load EvoTreeClassifier pkg=EvoTrees verbosity=0
kNNClassifier = MLJ.@load KNNClassifier pkg=NearestNeighborModels verbosity=0

In [ ]:
TrainBaseEnsembles(:AdaB, Dict("n_iter" => 100), (train_inputs, train_targets), cross_val_index, n_comp)

In [ ]:
TrainBaseEnsembles(:BaggingC, Dict("n_estimators" => 10), (train_inputs, train_targets), cross_val_index, n_comp)

In [ ]:
TrainBaseEnsembles(:EvoT, Dict("n_estimators" => 10), (train_inputs, train_targets), cross_val_index, n_comp)